In [ ]:
import pandas
from Dataset import Load_data
import tensorflow.keras as keras
from keras import layers
import tensorflow as tf
# Cấu hình sử dụng GPU
GPU=1
physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_visible_devices(physical_devices[GPU], 'GPU')

# Khởi tạo cấu hình GPU
tf.config.experimental.set_memory_growth(physical_devices[GPU], True)

In [2]:
class LSTM_Custom(keras.Model):
    def __init__(self):
        super(LSTM_Custom, self).__init__()
        self.lstm1 = layers.LSTM(512, return_sequences=True)
        self.lstm2= layers.Bidirectional(layers.LSTM(256,return_sequences=True))
        self.act=layers.Activation("relu")
        self.lstm3 = layers.Bidirectional(layers.LSTM(128))
        self.lstm3 = layers.Bidirectional(layers.LSTM(64))

    def call(self, inputs):
        x = self.lstm1(inputs)
        x= self.lstm2(x)
        x=self.act(x)
        x=self.lstm3(x)
        return x

class Prediction(keras.Model):
    def __init__(self):
        super(Prediction, self).__init__()
        self.dense1 = layers.Dense(128,activation="relu")
        self.dropout = layers.Dropout(0.3)
        self.predict = layers.Dense(48,activation="relu")

    def call(self, inputs):
        x = self.dense1(inputs)
        x=self.dropout(x)
        x=self.predict(x)
        return x

In [3]:
def CreateModel():
    inputs = keras.Input(shape=(24,24, 3))
    x=layers.Reshape((-1,72))(inputs)
    x=LSTM_Custom()(x)
    x=layers.Normalization()(x)
    x=Prediction()(x)
    model = keras.Model(inputs=inputs, outputs=x)
    return model

In [ ]:
model=CreateModel()
model.summary()
data_train,data_val=Load_data()()

In [ ]:
model.compile(loss='mean_absolute_error',optimizer=keras.optimizers.Adam(learning_rate=0.00001),metrics="accuracy")
model.load_weights('./LSTM_weights.h5')
with tf.device(f'/GPU:{GPU}'):
    model.fit(data_train, epochs=50,validation_data=data_val)
    model.save_weights('./LSTM_weights.h5')